In [10]:
import pandas as pd
import numpy as np
from brainpipe.system import study
from os import path
import os

st = study('Olfacto')
path_data = path.join(st.path,'database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_EpiScore_Expi/')
files = os.listdir(path_data)
#print(files)
for fi in files:
    if fi.startswith('MICP'):
        loadname = path_data+fi
        mat = np.load(loadname)
        x = np.load(loadname)['x']
        print (fi, x.shape, mat.files)


-> Olfacto loaded
MICP_odor_bad.npz (121, 2816, 33) ['x', 'xyz', 'sf', 'channel', 'label']
MICP_odor_bad_bipo.npz (104, 2816, 33) ['xyz', 'x', 'channel', 'sf', 'label']
MICP_odor_bad_bipo2.npz (100, 2816, 33) ['channel', 'x', 'sf', 'label', 'xyz']
MICP_odor_bad_bipo_new.npz (76, 2816, 45) ['label', 'xyz', 'x', 'sf', 'channel']
MICP_odor_bad_bipo_sel.npz (74, 2816, 45) ['channel', 'x', 'sf', 'label', 'xyz']
MICP_odor_bad_bipo_sel_aal_4.npz (60, 2816, 45) ['xyz', 'sf', 'labels', 'BA', 'channels', 'aal', 'x']
MICP_odor_bad_bipo_sel_aal_5.npz (45, 2816, 45) ['xyz', 'sf', 'labels', 'BA', 'channels', 'aal', 'x']
MICP_odor_bad_bipo_sel_phys.npz (67, 2816, 45) ['channels', 'BA', 'sf', 'labels', 'xyz', 'aal', 'x']
MICP_odor_bad_bipo_sel_phys2.npz (62, 2816, 45) ['sf', 'phys', 'x', 'labels', 'channels', 'xyz']
MICP_odor_good.npz (121, 2816, 45) ['x', 'xyz', 'sf', 'channel', 'label']
MICP_odor_good_bipo.npz (104, 2816, 45) ['xyz', 'x', 'channel', 'sf', 'label']
MICP_odor_good_bipo2.npz (100, 2816

## Remove bad electrodes after artefact detection
### TF with artefacts or 50Hz

In [ ]:
import numpy as np
from brainpipe.system import study
from os import path

In [7]:
st = study('Olfacto')
pathfiles = path.join(st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_EpiScore_Expi/')
subjects = ['MICP'] #CHAF','VACJ','SEMC','LEFC','MICP','PIRJ','FERJ'
conds = ['bad', 'good']
phases = ['odor']#'rest'
rej_elec = {
    'CHAF' : [98,102,103,104],
    'VACJ' : [0,66,67,68,122,128], 
    'SEMC' : [36],
    'PIRJ' : [10,14,15,25,26,41,47,48,53,54,55,57,58,63,64,71,72,79,80,81,82,83,],
    'LEFC' : [47,48,49,50,140,141,],
    'MICP' : [4,9,13,71,],
    'FERJ': [],
            }

for su in subjects:
    for cond in conds:
        for phase in phases:
            mat = np.load(pathfiles+su+'_'+phase+'_'+cond+'_bipo.npz')
            x, sf, label, channel, xyz = mat['x'], mat['sf'], mat['label'], mat['channel'], mat['xyz']
            print (su, x.shape, label.shape, channel.shape, xyz.shape)

            #Remove all bad electrodes from the database
            new_x = np.delete(x,rej_elec[su], axis=0)
            new_label = np.delete(label,rej_elec[su], axis=0)
            new_channel = np.delete(channel,rej_elec[su], axis=0)
            new_xyz = np.delete(xyz,rej_elec[su], axis=0)
            print (su, len(rej_elec[su]),new_x.shape, new_label.shape, new_channel.shape, new_xyz.shape)

            #Save all new information
            mat_new = {'x':new_x, 'label':new_label, 'channel':new_channel, 'xyz':new_xyz, 'sf':sf}
            file_source = su+'_'+phase+'_'+cond+'_bipo2.npz'
            np.savez(pathfiles+file_source,**mat_new)

-> Olfacto loaded
MICP (104, 2816, 33) (104,) (104,) (104, 3)
MICP 4 (100, 2816, 33) (100,) (100,) (100, 3)
MICP (104, 2816, 45) (104,) (104,) (104, 3)
MICP 4 (100, 2816, 45) (100,) (100,) (100, 3)


## Remove electrodes with WM

In [8]:
st = study('Olfacto')
pathfiles = path.join(st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_EpiScore_Expi/')
subjects = ['MICP'] #'CHAF','VACJ','SEMC', 'LEFC','MICP', 
phases = ['odor']
for su in subjects:
    for phase in phases:
        print(su)
        #load all subjects information
        badname = np.load(pathfiles+su+'_odor_bad_bipo2.npz')
        goodname = np.load(pathfiles+su+'_odor_good_bipo2.npz')
        bad, bad_label, bad_channel, bad_xyz = badname['x'], badname['label'], badname['channel'], badname['xyz']
        good, good_label, good_channel, good_xyz = goodname['x'], goodname['label'], goodname['channel'], goodname['xyz']
        sf = goodname['sf']
        print('bad shape: ', bad.shape, bad_label.shape, bad_channel.shape, bad_xyz.shape)
        print('good shape: ', good.shape, good_label.shape, good_channel.shape, good_xyz.shape)

        #create a list of all electrodes to remove
        elecs = bad_label.shape[0]
        print(elecs)
        elecs_rej = []
        for elec in range(elecs):
            elec_rej_wm = np.where(bad_label[elec].find('WM') != -1,int(elec),None)
            elec_rej_lcr = np.where(bad_label[elec].find('LCR') != -1,int(elec),None)
            elec_rej_skull = np.where(bad_label[elec].find('skull') != -1,int(elec),None)
            if elec_rej_wm != None:
                elecs_rej = np.append(elecs_rej,elec_rej_wm)
            if elec_rej_lcr != None and elec_rej_lcr != elec_rej_wm:
                elecs_rej = np.append(elecs_rej,elec_rej_lcr)
            if elec_rej_skull != None and elec_rej_skull != elec_rej_wm and elec_rej_skull != elec_rej_lcr:
                elecs_rej = np.append(elecs_rej,elec_rej_skull)
        print(su, elecs, elecs_rej)

        #Remove all bad electrodes from the database
        new_bad, new_bad_label = np.delete(bad,elecs_rej, axis=0), np.delete(bad_label,elecs_rej, axis=0)
        new_bad_channel, new_bad_xyz = np.delete(bad_channel,elecs_rej, axis=0), np.delete(bad_xyz,elecs_rej, axis=0)
        new_good, new_good_label = np.delete(good,elecs_rej, axis=0), np.delete(good_label,elecs_rej, axis=0)
        new_good_channel, new_good_xyz = np.delete(good_channel,elecs_rej, axis=0), np.delete(good_xyz,elecs_rej, axis=0)
        print (su,phase,'bad',new_bad.shape, new_bad_label.shape,new_bad_channel.shape, new_bad_xyz.shape,
              'good', new_good.shape, new_good_label.shape, new_good_channel.shape, new_good_xyz.shape)

        #Save all new information
        mat_good = {'x':new_bad, 'label':new_bad_label, 'channel':new_bad_channel, 'xyz':new_bad_xyz, 'sf':sf}
        mat_bad = {'x':new_good, 'label':new_good_label, 'channel':new_good_channel, 'xyz':new_good_xyz, 'sf':sf}
        file_good = su+'_'+phase+'_good_bipo_sel.npz'
        file_bad = su+'_'+phase+'_bad_bipo_sel.npz'
        np.savez(pathfiles+file_good,**mat_good)
        np.savez(pathfiles+file_bad,**mat_bad)
        del elecs, elecs_rej, new_bad, new_bad_label, new_bad_channel, new_bad_xyz, new_good
        del new_good_label, new_good_channel, new_good_xyz

-> Olfacto loaded
MICP
bad shape:  (100, 2816, 33) (100,) (100,) (100, 3)
good shape:  (100, 2816, 45) (100,) (100,) (100, 3)
100
MICP 100 [2 3 4 7 11 19 20 21 29 30 31 42 43 44 61 70 71 72 73 74 77 78 88 89 93 99]
MICP odor bad (74, 2816, 33) (74,) (74,) (74, 3) good (74, 2816, 45) (74,) (74,) (74, 3)


/home/karim/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:34: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
/home/karim/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:35: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
/home/karim/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:36: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
/home/karim/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:37: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


### Add AAL & BA labels to subjects' matrices

In [9]:
import pandas as pd
import numpy as np
from brainpipe.system import study
from os import path

st = study('Olfacto')
subjects = ['MICP'] #'LEFC','CHAF','VACJ','SEMC','FERJ','MICP','PIRJ'
#subjects = ['LEFC']
conds=['bad','good']
path_data = path.join(st.path,'database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_EpiScore_Expi/')
path_labels = path.join(path_data,'labels/')

for su in subjects:
    for cond in conds:
        # Load the data
        mat = np.load(path_data+su+'_odor_'+cond+'_bipo_sel.npz')
        kwargs = {}
        kwargs['x'], kwargs['channels'], kwargs['xyz'] = mat['x'], mat['channel'],mat['xyz']
        kwargs['labels'], kwargs['sf']=mat['label'],mat['sf']
        # Load the AAL labels
        #aal = su+'_physio_labels_aal.csv'
        aal = 'Elecs_'+su+'_labels_aal.csv'
        df = pd.read_csv(path_labels+aal, sep=',')
        kwargs['aal'] = df['aal'].values
        # Load the BA labels
        BA = 'Elecs_'+su+'_labels_brodmann.csv'
        df = pd.read_csv(path_labels+BA, sep=',')
        kwargs['BA'] = df['brodmann'].values
        # Update the datafile
        filename2 = su+'_odor_'+cond+'_bipo_sel_phys.npz'
        np.savez(path_data+filename2, **kwargs)
        mat2 = np.load(path_data+su+'_odor_'+cond+'_bipo_sel_phys.npz')
        print(mat2.files)
        print(su,mat2['x'].shape, mat2['xyz'].shape, mat2['aal'].shape, mat2['BA'].shape)
    

-> Olfacto loaded


FileNotFoundError: File b'/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_EpiScore_Expi/labels/Elecs_MICP_labels_aal.csv' does not exist

### Select only regions present in at least 2 patients 
#### Remove all not memory related regions (motor, precentral, SMA)

In [ ]:
st = study('Olfacto')
pathfiles = path.join(st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_EpiScore_Expi/')
subjects = ['CHAF','VACJ','SEMC','LEFC','MICP','PIRJ','FERJ']
conds = ['bad','good']
to_keep = ['Amygdala (R)', 'Cingulum Ant (R)','Cingulum Mid (R)', 'Frontal Inf Orb (R)', 
           'Frontal Inf Tri (R)', 'Frontal Mid (R)', 'Frontal Mid Orb (R)', 'Frontal Sup (R)',
           'Frontal Sup Medial (R)', 'Frontal Sup Orb (R)', 'Hippocampus (R)', 'Insula (R)',
           'ParaHippocampal (R)','Temporal Inf (R)', 'Temporal Mid (R)', 'Temporal Pole Mid (R)',
           'Temporal Pole Sup (R)', 'Temporal Sup (R)','Amygdala (L)', 'Cingulum Ant (L)', 
           'Cingulum Mid (L)', 'Frontal Inf Orb (L)', 'Frontal Inf Tri (L)','Frontal Mid (L)', 
           'Frontal Mid Orb (L)', 'Frontal Sup (L)','Frontal Sup Medial (L)', 'Frontal Sup Orb (L)',
           'Hippocampus (L)', 'Insula (L)','ParaHippocampal (L)','Temporal Inf (L)', 
           'Temporal Mid (L)','Temporal Pole Mid (L)','Temporal Pole Sup (L)', 'Temporal Sup (L)']

for su in subjects:
    for cond in conds:
        mat = np.load(pathfiles+su+'_odor_'+cond+'_bipo_sel_phys.npz')
        idx = np.where([mat['phys'][i] in to_keep for i in range(len(mat['phys']))])
        kwargs = {}
        kwargs['labels'], kwargs['x'], kwargs['phys'] = mat['labels'][idx], mat['x'][idx], mat['phys'][idx]
        kwargs['xyz'], kwargs['channels'], kwargs['sf'] = mat['xyz'][idx], mat['channels'][idx], mat['sf']
        np.savez(pathfiles+su+'_odor_'+cond+'_bipo_sel_phys2.npz',**kwargs)
        del mat